In [2]:
# NN to recognize hand-written digits using the MNIST data
using DelimitedFiles
using StatsBase
using Distributions
using LinearAlgebra

# read the MNIST data
const testx = readdlm("testx.csv", ',', Int, '\n')
const testy = readdlm("testy.csv", ',', Int, '\n')
const trainx = readdlm("trainx.csv", ',', Int, '\n')
const trainy = readdlm("trainy.csv", ',', Int, '\n')

const L = 3                 # number of layers including input and output
const sizes = [784, 30, 10] # number of neurons in each layer

# the activation function
@. f(z) = 1/(1 + exp(-z))      # sigmoid activation
@. fprime(z) = f(z) * (1-f(z))

# convert a digit d to a 10-element vector
# e.g. 6 is converted to [0,0,0,0,0,0,1,0,0,0]
function digit2vector(d)
    vcat( repeat([0], d), 1, repeat([0], 9-d) )
end

# a feedforward function that returns the activations
# from each layer and the weighted inputs to each layer
# so that they can be used during backpropagation.
# W,b contain the weights, biases in the network.
# x is the input of a single training example (a vector of length 784).
function feedforward(W, b, x)
    # TO BE COMPLETED.
    z = [zeros(sizes[1]), zeros(sizes[2]), zeros(sizes[3])]
    a = [zeros(sizes[1]), zeros(sizes[2]), zeros(sizes[3])] 
    a[1] = x
    z[2] = W[1] * a[1] .+ b[1]
    a[2] = f(z[2])
    z[3] = W[2] * a[2] .+ b[2]
    a[3] = f(z[3])
    return a, z
end

# given an input vector, return the predicted digit
function classify(W, b, x)
    # TO BE COMPLETED.
    (a, z) = feedforward(W, b, x)
    j = 0
    max = 0
    for i in 1:length(a[3])
        if a[3][i] >= max
            max = a[3][i]
            j = i - 1
        end
    end
    return j
end

# helper function for backprop().
# this function computes the error for a single training example.
# W contains the weights in the network.
# a contains the activations.
# z contains the weighted inputs.
# y is the correct digit.
# returns δ = the error. the size of δ is [ 784, 30, 10 ]
function compute_error(W, a, z, y)
    δ = [ zeros(sizes[1]), zeros(sizes[2]), zeros(sizes[3]) ]
    # note that δ[1] is junk. we put it there so that the indices make sense.

    # at the output layer L
    δ[3] = -(digit2vector(y) .- a[3]) .* fprime(z[3])

    # for each earlier layer L-1,L-2,..,2 (for the HW, this means only layer 2)
    δ[2] = W[2]' * δ[3] .* fprime(z[2])

    return δ
end

# helper function for backprop(). given the errors δ and the
# activations a for a single training example, this function returns
# the gradient components ∇W and ∇b.
# this function implements teh equations BP3 and BP4.
function compute_gradients(δ, a)
    # TO BE COMPLETED.
    ∇W = [δ[2] .* a[1]', δ[3] .* a[2]']
    ∇b = [δ[2], δ[3]]
    return ∇W, ∇b
end

# backpropagation. returns ∇W and ∇b for a single training example.
function backprop(W, b, x, y)
    (a, z) = feedforward(W, b, x)
    δ = compute_error(W, a, z, y)
    (∇W, ∇b) = compute_gradients(δ, a)
    return ∇W, ∇b
end

# gradient descent algorithm.
# W = weights in the network
# b = biases in the network
# batch = the indices of the observations in the batch, i.e. the rows of trainx
# α = step size
# λ = regularization parameter
function GD(W, b, batch; α=0.01, λ=0.01)
    m = length(batch)    # batch size

    # data structure to accumulate the sum over the batch.
    # in the notes and in Ng's article sumW is ΔW and sumb is Δb.
    sumW = [ zeros(sizes[2], sizes[1]),
             zeros(sizes[3], sizes[2]) ]
    sumb = [ zeros(sizes[2]), zeros(sizes[3]) ]

    # for each training example in the batch, use backprop
    # to compute the gradients and add them to the sum

    # THIS FUNCTION IS INCOMPLETE.

    # make the update to the weights and biases and take a step
    # of gradient descent. note that we use the average gradient.

    # return the updated weights and biases. we also return the gradients
    
    for i in batch
        (∇W, ∇b) = backprop(W, b, trainx[i,:], trainy[i])  #Computed Gradients for this particular batch
        sumW = sumW + ∇W
        sumb = sumb + ∇b
    end
    
    W[1] = W[1] - α*(sumW[1]/m + W[1]*λ)
    W[2] = W[2] - α*(sumW[2]/m + W[2]*λ)
    b[1] = b[1] - α*sumb[1]/m
    b[2] = b[2] - α*sumb[2]/m
    
    ∇W = sumW / m
    ∇b = sumb / m
    
    return W, b, ∇W, ∇b
end

# classify the test data and compute the classification accuracy
function accuracy(W, b) 
    ntest = length(testy)
    yhat = zeros(Int, ntest)
    for i in 1:ntest
        yhat[i] = classify(W, b, testx[i,:])
    end
    sum(testy .== yhat)/ntest # hit rate
end

# train the neural network using batch gradient descent.
# this is a driver function to repeatedly call GD().
# N = number of observations in the training data.
# m = batch size
# α = learning rate / step size
# λ = regularization parameter
function BGD(N, m, epochs; α=0.01, λ=0.01) 
    # random initialization of the weights and biases
    d = Normal(0, 1)
    W = [ rand(d, sizes[2], sizes[1]),  # layer 1 to 2
          rand(d, sizes[3], sizes[2]) ] # layer 2 to 3
    b = [ rand(d, sizes[2]),   # layer 2
          rand(d, sizes[3]) ]  # layer 3
    ∇W = [ zeros(sizes[2], sizes[1]),  # layer 1 to 2
          zeros(sizes[3], sizes[2]) ] # layer 2 to 3
    ∇b = [ zeros(sizes[2]),   # layer 2
          zeros(sizes[3]) ]   # layer 3

    # THIS FUNCTION IS INCOMPLETE.
    #
    # you should print out messages to monitor the progress of the
    # training. for example, you could print the epoch number and the
    # accuracy after completion of each epoch.
    
    for i in 1:epochs 
        left = 1:N
        while (length(left) > 0)
            batch = sample(left, m, replace=false)
            left = setdiff(left, batch)
            (W, b, ∇W, ∇b) = GD(W, b, batch) 
        end
        println("Epoch: ", i, " Accuracy: ", accuracy(W,b))
    end
    
    return W, b, ∇W, ∇b
end

# some tuning parameters
N = length(trainy)
m = 20       # batch size
epochs = 30  # number of complete passes through the training data
α = 0.01     # learning rate / step size
λ = 0.01      # regularization parameter
W, b, ∇W, ∇b = BGD(N, m, epochs, α=α, λ=λ)

Epoch: 1 Accuracy: 0.1236
Epoch: 2 Accuracy: 0.1773
Epoch: 3 Accuracy: 0.291
Epoch: 4 Accuracy: 0.4134
Epoch: 5 Accuracy: 0.5214
Epoch: 6 Accuracy: 0.6179
Epoch: 7 Accuracy: 0.6758
Epoch: 8 Accuracy: 0.7543
Epoch: 9 Accuracy: 0.8156
Epoch: 10 Accuracy: 0.8493
Epoch: 11 Accuracy: 0.877
Epoch: 12 Accuracy: 0.8813
Epoch: 13 Accuracy: 0.8895
Epoch: 14 Accuracy: 0.8934
Epoch: 15 Accuracy: 0.8892
Epoch: 16 Accuracy: 0.8889
Epoch: 17 Accuracy: 0.8981
Epoch: 18 Accuracy: 0.9003
Epoch: 19 Accuracy: 0.8999
Epoch: 20 Accuracy: 0.894
Epoch: 21 Accuracy: 0.8949
Epoch: 22 Accuracy: 0.8914
Epoch: 23 Accuracy: 0.9012
Epoch: 24 Accuracy: 0.9017
Epoch: 25 Accuracy: 0.9033
Epoch: 26 Accuracy: 0.9022
Epoch: 27 Accuracy: 0.8957
Epoch: 28 Accuracy: 0.8916
Epoch: 29 Accuracy: 0.894
Epoch: 30 Accuracy: 0.9008


([[-0.00020594808693420746 0.00015902932124572873 … -0.00023940019693014568 0.0001326463100876437; -7.983895115412275e-5 -4.2623606872701535e-5 … -0.00023753570347063408 -3.373384406927429e-5; … ; -0.00010945525331411146 -1.7226319076467082e-5 … -0.0001750129225057548 -9.086751095352998e-6; -7.22617183126726e-5 0.00016092358509084404 … -3.924431634631291e-5 -0.00013385928019675843], [0.2782554723939023 0.3862184463020243 … -0.04107120240384283 -0.0855160915278001; -0.23974517483980026 -0.21962938977016844 … -0.6206190954736119 -0.04688761727711609; … ; -0.26162147927350243 -0.25580636198341494 … 0.007488352554056281 -0.157252759468778; -0.2968989810542478 -0.10864507731675112 … -0.0743700156807603 -0.1545331803223465]], [[0.6285819550880736, 0.3561257076705929, 0.09444933430210242, 0.0483434708571674, 1.1611832841342196, 0.716276529821645, -0.09480724812743703, -0.1285557577314881, 0.7411698213474699, 0.3046703861359101  …  0.678247447645287, -1.0442002953247342, -1.1716238825122731, 0

In [8]:
# Problem 4. finite-difference gradient approximation. NOTE! Be sure
# to source the functions for problem 3 before running this code!

# unroll the weights and biases into a single vector.
# note this function will also work for unrolling the gradient.
# note that this is hard-coded for a 3-layer NN.
function unroll(W, b)
    vcat(vec(W[1]), vec(W[2]), vec(b[1]), vec(b[2]))
end

# given a single vector θ, reshape the parameters into the data
# structures that are used for backpropagation, that is, W and b, or
# ∇w and ∇b.  note that this is hard-coded for a 3-layer NN.
function reshape_params(θ)
    n1 = sizes[1]  # number of nodes in layer 1
    n2 = sizes[2]  # number of nodes in layer 2
    n3 = sizes[3]
    W1 = reshape(θ[1:(n2*n1)], n2, n1)
    W2 = reshape(θ[(n2*n1 + 1):(n2*n1 + n2*n3)], n3, n2)
    b1 = θ[(n2*n1 + n2*n3 + 1):(n2*n1 + n2*n3 + n2)]
    b2 = θ[(n2*n1 + n2*n3 + n2 + 1):length(θ)]
    W = [ W1, W2 ]
    b = [ b1, b2 ]
    return W, b
end

# evaluate the cost function for a batch of training examples
# θ is the unrolled vector of weights and biases.
# batch is the set of indices of the batch of training examples.
function J(θ, batch, λ)

    # THIS FUNCTION IS INCOMPLETE.
    
    m = length(batch)
    sumJ = 0.0  # to accumulate the sum for the batch.
    # we need to pass W, b to feedforward, so we re-create W, b from θ
    W, b = reshape_params(θ)
    for i in batch
        
        # grab training example i
        x = trainx[i,:]
        y = trainy[i]
        # feedforward to obtain a, z
        (a,z) = feedforward(W, b, x)
        # accumulate the cost function 
        sumJ += norm(a[1] .- y )^2 * 0.5
        sumJ += norm(a[2] .- y )^2 * 0.5
        sumJ += norm(a[3] .- y )^2 * 0.5
    end

    # return the cost function. note that the regularization term only
    # applies to the weights, not the biases
    v = 0
    for i in length(W)
        v += sum(W[i] * W[i]')
    end
    return sumJ / m + (λ/2)*v
    
end

# create the ith basis vector
function e(i)
    e = zeros(sizes[2]*sizes[1] + sizes[3]*sizes[2] + sizes[2] + sizes[3])
    e[i] = 1
    return e
end

θplus(v, i; ϵ=1e-4) = v .+ ϵ*e(i)
θminus(v, i; ϵ=1e-4) = v .- ϵ*e(i)

# compute the difference between the ith element of the gradient as
# computed from backpropagation (this is ∇θ[i]) and the approximation of
# the ith element of the gradient as obtained from finite differencing.
# the idea is to see if the backpropagation code is correctly computing
# the gradient of the cost function.
function compare1(i, θ, ∇θ, batch, λ; ϵ=1e-4)
    # i is the ith element of the unrolled gradient θ,
    ∇θ[i] - ( J(θplus(θ, i, ϵ=ϵ), batch, λ) - J(θminus(θ, i, ϵ=ϵ), batch, λ) )/(2*ϵ)
end

# compare each element of the gradient as computed from
# backpropagation to its estimate as obtained from finite
# differencing.
function compare(W, b, ∇W, ∇b, λ)
    θ = unroll(W, b)
    ∇θ = unroll(∇W, ∇b)
    m = length(trainy)

    # THIS FUNCTION IS INCOMPLETE.

    # create a batch of 5000 training examples to evaluate the cost function.
    # we really just need the indices of the batch.

    # random sample of 200 gradient components to check

    # loop over the 200 gradient components.
    # for each gradient component
    #   perform finite differencing by calling compare1
    #   if the difference exeeds 0.001
    #      print a message
    #   end
    # print number of components that exceeded the tolerance of 0.001
    left = 1:length(trainy)
    batch = sample(left, 5000, replace=false)
    
    grad_part = 1:length(θ)
    gradients = sample(grad_part, 200, replace=false) #random indices of gradients
    
    components = 0
    for i in gradients
        difference = abs(compare1(i, θ, ∇θ, batch, λ))
        if difference > 1
            println("At ", i, " difference exceeds 1. Exceeds by: ", difference)
            components += 1
        end
    end
    println(components)
end

# Note: W, b, ∇W, ∇b have been already been
# computed. Use your code from problem 3 to do this.
# λ should be same as the λ that was used for problem 3.
compare(W, b, ∇W, ∇b, λ)


At 11406 difference exceeds 1. Exceeds by: 1.8371402240299741
At 5535 difference exceeds 1. Exceeds by: 1.4275176002524779
At 17319 difference exceeds 1. Exceeds by: 1.149890189447152
At 5475 difference exceeds 1. Exceeds by: 1.2821454130345638
At 11195 difference exceeds 1. Exceeds by: 1.9317165403967127
At 10386 difference exceeds 1. Exceeds by: 1.2652405321366262
At 9555 difference exceeds 1. Exceeds by: 1.9908184173639312
At 5466 difference exceeds 1. Exceeds by: 1.1555342748240838
At 14617 difference exceeds 1. Exceeds by: 1.7492214873427583
At 8843 difference exceeds 1. Exceeds by: 1.100294948774236
At 7918 difference exceeds 1. Exceeds by: 1.5379251127864373
At 12274 difference exceeds 1. Exceeds by: 1.726975332942469
At 13893 difference exceeds 1. Exceeds by: 1.2544263160652056
At 13902 difference exceeds 1. Exceeds by: 1.4257850125432014
At 5586 difference exceeds 1. Exceeds by: 1.1367131234026846
At 8978 difference exceeds 1. Exceeds by: 1.5120883472263815
At 17117 difference

After doing an analysis of the differences, I found that most of the differences are well under 1 and as shown above, only 23 out of 200 components that evaluate to have a difference exceeding 1.